In [36]:
import MetaTrader5 as mt5
import pandas as pd

In [37]:
mt5.initialize()

True

In [38]:
mt5.last_error()

(1, 'Success')

In [39]:
login = 88548079
password = '0fPnWs-b'
server = 'MetaQuotes-Demo'

mt5.login(login, password, server)

True

In [40]:
mt5.account_info()

AccountInfo(login=88548079, trade_mode=0, leverage=100, limit_orders=200, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=10000.0, credit=0.0, profit=0.0, equity=10000.0, margin=0.0, margin_free=10000.0, margin_level=0.0, margin_so_call=50.0, margin_so_so=30.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='Dananjaya Kalhara', server='MetaQuotes-Demo', currency='USD', company='MetaQuotes Ltd.')

In [41]:
symbol = mt5.symbol_info("XAUUSD")
symbol.description

'Gold vs US Dollar'

In [42]:
def market_order(symbol, volume, order_type, deviation=20, magic=15, stoploss=0.0, strategy_name='Trading Bot'):
    order_type_dict = {
        'buy': mt5.ORDER_TYPE_BUY,
        'sell': mt5.ORDER_TYPE_SELL
    }

    price_dict = {
        'buy': mt5.symbol_info_tick(symbol).ask,
        'sell': mt5.symbol_info_tick(symbol).bid
    }

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,  # FLOAT
        "type": order_type_dict[order_type],
        "price": price_dict[order_type],
        "sl": stoploss,  # FLOAT
        "tp": 0.0,  # FLOAT
        "deviation": deviation,  # INTERGER
        "magic": magic,  # INTERGER
        "comment": strategy_name,
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    order_result = mt5.order_send(request)
    return (order_result)


def close_position(position, deviation=20, magic=15, strategy_name='Trading Bot'):
    order_type_dict = {
        0: mt5.ORDER_TYPE_SELL,
        1: mt5.ORDER_TYPE_BUY
    }

    price_dict = {
        0: mt5.symbol_info_tick(position['symbol']).bid,
        1: mt5.symbol_info_tick(position['symbol']).ask
    }

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "position": position['ticket'],  # select the position you want to close
        "symbol": position['symbol'],
        "volume": position['volume'],  # FLOAT
        "type": order_type_dict[position['type']],
        "price": price_dict[position['type']],
        "deviation": deviation,  # INTERGER
        "magic": magic,  # INTERGER
        "comment": strategy_name,
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    order_result = mt5.order_send(request)
    return (order_result)


def close_all_positions(order_type, magic=15):
    order_type_dict = {
        'buy': 0,
        'sell': 1
    }

    if mt5.positions_total() > 0:
        positions = mt5.positions_get()

        positions_df = pd.DataFrame(positions, columns=positions[0]._asdict().keys())
        positions_df = positions_df[positions_df['magic'] == magic]

        if order_type != 'all':
            positions_df = positions_df[(positions_df['type'] == order_type_dict[order_type])]

        for i, position in positions_df.iterrows():
            order_result = close_position(position)

            print('order_result: ', order_result)



In [47]:
market_order(symbol.name,0.01,'buy')

OrderSendResult(retcode=10009, deal=3072790821, order=3112059062, volume=0.01, price=2645.38, bid=2645.23, ask=2645.38, comment='Request executed', request_id=2211919257, retcode_external=0, request=TradeRequest(action=1, magic=15, order=0, symbol='XAUUSD', volume=0.01, price=2645.38, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=0, type_filling=1, type_time=0, expiration=0, comment='Trading Bot', position=0, position_by=0))

In [ ]:
OrdersBuyPrices = []
OrdersSellPrices = []
GridSize = 1.0
Volume = 0.01

nextSellOrderTrigerPrice = 0.00
nextBuyOrderTrigerPrice = 0.00

lastOrderPrice = symbol.ask
nextBuyOrderTrigerPrice = symbol.ask
while True:
 
    if symbol.ask >= nextBuyOrderTrigerPrice:
        market_order(symbol.name,Volume,'buy')
        lastOrderPrice = nextBuyOrderTrigerPrice
        if nextBuyOrderTrigerPrice not in OrdersBuyPrices:
            OrdersBuyPrices.append(nextBuyOrderTrigerPrice)
        i = 0
        while True:
            nextBuyOrderTrigerPrice = lastOrderPrice + GridSize * i
            if nextBuyOrderTrigerPrice in OrdersBuyPrices:
                i = i + 1
            else:
                break   

    if symbol.ask <= nextSellOrderTrigerPrice:
        market_order(symbol.name,Volume,'sell')
        lastOrderPrice = nextSellOrderTrigerPrice
        if nextSellOrderTrigerPrice not in OrdersSellPrices:
            OrdersSellPrices.append(nextSellOrderTrigerPrice)
        i = 0
        while True:
            nextSellOrderTrigerPrice = lastOrderPrice + GridSize * i
            if nextSellOrderTrigerPrice in OrdersSellPrices:
                i = i + 1
            else:
                break           
    
    print(f'\rNext Buy: {nextBuyOrderTrigerPrice}\nNext Sell: {nextSellOrderTrigerPrice}',end='')
